# Factorizations and other fun
Based on work by Andreas Noack

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system

In [13]:
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Array{Float64,1}:
 2.02826
 1.40671
 1.91034

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA=LU
``` 
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [14]:
Alu = lufact(A)

Base.LinAlg.LU{Float64,Array{Float64,2}} with factors L and U:
[1.0 0.0 0.0; 0.284734 1.0 0.0; 0.862877 0.550833 1.0]
[0.762028 0.143738 0.500939; 0.0 0.82268 0.805042; 0.0 0.0 -0.200082]

In [15]:
typeof(Alu)

Base.LinAlg.LU{Float64,Array{Float64,2}}

The different parts of the factorization can be extracted by special indexing

In [16]:
Alu[:P]

3×3 Array{Float64,2}:
 0.0  1.0  0.0
 1.0  0.0  0.0
 0.0  0.0  1.0

In [17]:
Alu[:L]

3×3 Array{Float64,2}:
 1.0       0.0       0.0
 0.284734  1.0       0.0
 0.862877  0.550833  1.0

In [18]:
Alu[:U]

3×3 Array{Float64,2}:
 0.762028  0.143738   0.500939
 0.0       0.82268    0.805042
 0.0       0.0       -0.200082

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [19]:
A\b

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

In [20]:
Alu\b

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [21]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
``` 

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`. 

In [22]:
Aqr = qrfact(A)

Base.LinAlg.QRCompactWY{Float64,Array{Float64,2}} with factors Q and R:
[-0.210733 0.887246 -0.41035; -0.740105 -0.419046 -0.52597; -0.63862 0.192862 0.744962]
[-1.02962 -0.656975 -1.00191; 0.0 0.817317 0.761205; 0.0 0.0 -0.149053]

The matrices `Q` and `R` can be extracted from the QR factorization object via

In [23]:
Aqr[:Q]

3×3 Base.LinAlg.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.210733   0.887246  -0.41035 
 -0.740105  -0.419046  -0.52597 
 -0.63862    0.192862   0.744962

In [24]:
Aqr[:R]

3×3 Array{Float64,2}:
 -1.02962  -0.656975  -1.00191 
  0.0       0.817317   0.761205
  0.0       0.0       -0.149053

In this case, R is stored as a 2x2 matrix rather than as a 3x2 because the last row of R is filled with 0's.<br>

Even though the matrix `Aqr[:Q]` is printed as a $3\times 3$ matrix in the factorization object, in practice it can represent the thin version as well (Here, a $3\times2$ matrix).

Julia infers if we want to work with the thin version

In [25]:
Aqr[:Q]*ones(2)

LoadError: [91mDimensionMismatch("vector must have length either 3 or 3")[39m

or the full version of Q

In [26]:
Aqr[:Q]*ones(3)

3-element Array{Float64,1}:
  0.266163
 -1.68512 
  0.299204

#### Eigendecompositions and the SVD(s)

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [27]:
Asym = A + A'
AsymEig = eigfact(Asym)

Base.LinAlg.Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}([-1.33487, -0.203179, 3.61069], [0.755599 -0.304306 0.580058; -0.623315 -0.606253 0.493899; -0.201365 0.734749 0.647763])

The values and the vectors can be extracted from the Eigen type by special indexing

In [28]:
AsymEig[:values]

3-element Array{Float64,1}:
 -1.33487 
 -0.203179
  3.61069 

In [29]:
AsymEig[:vectors]

3×3 Array{Float64,2}:
  0.755599  -0.304306  0.580058
 -0.623315  -0.606253  0.493899
 -0.201365   0.734749  0.647763

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [30]:
inv(AsymEig)*AsymEig

LoadError: [91mMethodError: no method matching *(::Array{Float64,2}, ::Base.LinAlg.Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}})[0m
Closest candidates are:
  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:424
  *(::Union{Base.ReshapedArray{T<:Union{Complex{Float32}, Complex{Float64}, Float32, Float64},2,A,MI} where MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N} where N} where A<:Union{DenseArray, SubArray{T,N,P,I,true} where I<:Tuple{Union{Base.Slice, UnitRange},Vararg{Any,N} where N} where P where N where T}, DenseArray{T<:Union{Complex{Float32}, Complex{Float64}, Float32, Float64},2}, SubArray{T<:Union{Complex{Float32}, Complex{Float64}, Float32, Float64},2,A,I,L} where L} where I<:Tuple{Vararg{Union{Base.AbstractCartesianIndex, Int64, Range{Int64}},N} where N} where A<:Union{Base.ReshapedArray{T,N,A,MI} where MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N} where N} where A<:Uni

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [31]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [32]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [33]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

-0.7413653846058953

In [34]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [35]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [36]:
@time eigvals(Asym);

  0.393044 seconds (18.08 k allocations: 8.840 MiB)


In [37]:
@time eigvals(Asym_noisy);

  1.452718 seconds (24 allocations: 7.921 MiB)


In [38]:
@time eigvals(Asym_explicit);

  0.213600 seconds (1.86 k allocations: 8.088 MiB)


In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` type to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [39]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  1.319335 seconds (77.37 k allocations: 187.367 MiB, 11.47% gc time)


6.898746295183322

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [40]:
1//2

1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [41]:
Arational = convert(Matrix{Rational{BigInt}}, rand(1:10,3,3))/10

3×3 Array{Rational{BigInt},2}:
 1//10  4//5  1//1 
 1//2   1//1  4//5 
 7//10  1//2  1//10

In [42]:
x = fill(1, (3))
b = Arational*x

3-element Array{Rational{BigInt},1}:
 19//10
 23//10
 13//10

In [43]:
Arational\b

3-element Array{Rational{BigInt},1}:
 1//1
 1//1
 1//1

In [46]:
lufact(Arational)

Base.LinAlg.LU{Rational{BigInt},Array{Rational{BigInt},2}} with factors L and U:
Rational{BigInt}[1//1 0//1 0//1; 5//1 1//1 0//1; 7//1 17//10 1//1]
Rational{BigInt}[1//10 4//5 1//1; 0//1 -3//1 -21//5; 0//1 0//1 6//25]

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```

In [52]:
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
Aeig = eigfact(A)
λ = Aeig[:values]

5-element Array{Float64,1}:
 -128.493 
  -55.8878
   42.7522
   87.1611
  542.468 

#### 11.2 
Create a `Diagonal` matrix from the eigenvalues of `A`.

In [53]:
Λ = Diagonal(λ)

5×5 Diagonal{Float64}:
 -128.493     ⋅        ⋅        ⋅         ⋅   
     ⋅     -55.8878    ⋅        ⋅         ⋅   
     ⋅        ⋅      42.7522    ⋅         ⋅   
     ⋅        ⋅        ⋅      87.1611     ⋅   
     ⋅        ⋅        ⋅        ⋅      542.468

#### 11.3 
Create a `LowerTriangular` matrix from `A`.

In [54]:
Alo = LowerTriangular(A)

5×5 LowerTriangular{Int64,Array{Int64,2}}:
 140    ⋅    ⋅    ⋅   ⋅
  97  106    ⋅    ⋅   ⋅
  74   89  152    ⋅   ⋅
 168  131  144   54   ⋅
 131   36   71  142  36